In [52]:
import pandas as pd
import catboost as cat
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


<h1>Simple Catboost</h1>

In [7]:
print(train_df.price.median())
print(train_df.image_top_1.max())
print(train_df.image_top_1.min())
train_df.isnull().any()

1300.0
3066.0
0.0


item_id                 False
user_id                 False
region                  False
city                    False
parent_category_name    False
category_name           False
param_1                  True
param_2                  True
param_3                  True
title                   False
description              True
price                    True
item_seq_number         False
activation_date         False
user_type               False
image                    True
image_top_1              True
deal_probability        False
dtype: bool

In [48]:
simple_df = train_df[['region', 'city', 'parent_category_name', 'category_name', 'price', 
                      'item_seq_number', 'user_type', 'image_top_1', 'deal_probability']]
simple_df.price = simple_df.price.fillna(simple_df.price.median())
simple_df.image_top_1 = simple_df.image_top_1.fillna(simple_df.image_top_1.max()+1)
simple_df.image_top_1 = simple_df.image_top_1.astype(int)
# simple_df = simple_df[:int(1e6)-1]
y = simple_df.deal_probability
simple_df = simple_df.drop(['deal_probability'], axis = 1)
print(simple_df.shape[0])
simple_df.isnull().any()

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


1503424


region                  False
city                    False
parent_category_name    False
category_name           False
price                   False
item_seq_number         False
user_type               False
image_top_1             False
dtype: bool

In [49]:
X_train, X_test, y_train, y_test = train_test_split(simple_df, y, test_size = 0.2, 
                                                    shuffle = False, random_state = 57)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size = 0.2, 
                                                    shuffle = False, random_state = 57)
cat_features = [0, 1, 2, 3, 6, 7]

In [ ]:
kitty = cat.CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=2,
        bagging_temperature=1,
        random_strength=5,
        random_seed=57,
        logging_level='Silent',
        use_best_model=True,
        calc_feature_importance=True,
#         custom_loss = ['Accuracy', 'AUC'],
        od_type='Iter',
        od_wait=10
    )
kitty.fit(
    X_train,
    y_train,
    cat_features,
    eval_set=(X_valid, y_valid),
    verbose = True
)

In [50]:
pred = kitty.predict(X_test)
# print(roc_auc_score(pred, y_test))

0:	learn: 0.2920023	test: 0.2923687	best: 0.2923687 (0)	total: 2.17s	remaining: 1h 12m 10s
1:	learn: 0.2892823	test: 0.2896654	best: 0.2896654 (1)	total: 3.66s	remaining: 1h 1m
2:	learn: 0.2867029	test: 0.2871003	best: 0.2871003 (2)	total: 4.98s	remaining: 55m 14s
3:	learn: 0.2842499	test: 0.2846628	best: 0.2846628 (3)	total: 6.81s	remaining: 56m 37s
4:	learn: 0.2819213	test: 0.2823478	best: 0.2823478 (4)	total: 8.37s	remaining: 55m 40s
5:	learn: 0.2797177	test: 0.2801557	best: 0.2801557 (5)	total: 9.8s	remaining: 54m 15s
6:	learn: 0.2776160	test: 0.2780671	best: 0.2780671 (6)	total: 11.5s	remaining: 54m 35s
7:	learn: 0.2756260	test: 0.2760890	best: 0.2760890 (7)	total: 12.9s	remaining: 53m 25s
8:	learn: 0.2737352	test: 0.2742092	best: 0.2742092 (8)	total: 13.9s	remaining: 51m 7s
9:	learn: 0.2718298	test: 0.2723011	best: 0.2723011 (9)	total: 15.5s	remaining: 51m 30s
10:	learn: 0.2700032	test: 0.2704728	best: 0.2704728 (10)	total: 17.1s	remaining: 51m 24s
11:	learn: 0.2682650	test: 0.26

92:	learn: 0.2348503	test: 0.2348561	best: 0.2348561 (92)	total: 2m 51s	remaining: 58m 38s
93:	learn: 0.2348136	test: 0.2348174	best: 0.2348174 (93)	total: 2m 53s	remaining: 58m 38s
94:	learn: 0.2347774	test: 0.2347795	best: 0.2347795 (94)	total: 2m 55s	remaining: 58m 39s
95:	learn: 0.2347248	test: 0.2347263	best: 0.2347263 (95)	total: 2m 57s	remaining: 58m 47s
96:	learn: 0.2346752	test: 0.2346760	best: 0.2346760 (96)	total: 3m	remaining: 58m 58s
97:	learn: 0.2346388	test: 0.2346373	best: 0.2346373 (97)	total: 3m 2s	remaining: 59m 1s
98:	learn: 0.2345773	test: 0.2345727	best: 0.2345727 (98)	total: 3m 4s	remaining: 59m 10s
99:	learn: 0.2345309	test: 0.2345263	best: 0.2345263 (99)	total: 3m 7s	remaining: 59m 16s
100:	learn: 0.2344844	test: 0.2344785	best: 0.2344785 (100)	total: 3m 9s	remaining: 59m 18s
101:	learn: 0.2344266	test: 0.2344141	best: 0.2344141 (101)	total: 3m 11s	remaining: 59m 21s
102:	learn: 0.2343729	test: 0.2343594	best: 0.2343594 (102)	total: 3m 13s	remaining: 59m 22s
10

181:	learn: 0.2325223	test: 0.2322940	best: 0.2322940 (181)	total: 5m 58s	remaining: 59m 40s
182:	learn: 0.2325032	test: 0.2322708	best: 0.2322708 (182)	total: 6m	remaining: 59m 43s
183:	learn: 0.2324935	test: 0.2322597	best: 0.2322597 (183)	total: 6m 3s	remaining: 59m 43s
184:	learn: 0.2324792	test: 0.2322453	best: 0.2322453 (184)	total: 6m 5s	remaining: 59m 42s
185:	learn: 0.2324688	test: 0.2322346	best: 0.2322346 (185)	total: 6m 7s	remaining: 59m 41s
186:	learn: 0.2324601	test: 0.2322255	best: 0.2322255 (186)	total: 6m 8s	remaining: 59m 34s
187:	learn: 0.2324465	test: 0.2322115	best: 0.2322115 (187)	total: 6m 10s	remaining: 59m 34s
188:	learn: 0.2324374	test: 0.2322020	best: 0.2322020 (188)	total: 6m 12s	remaining: 59m 31s
189:	learn: 0.2324220	test: 0.2321855	best: 0.2321855 (189)	total: 6m 14s	remaining: 59m 31s
190:	learn: 0.2324151	test: 0.2321795	best: 0.2321795 (190)	total: 6m 16s	remaining: 59m 29s
191:	learn: 0.2324013	test: 0.2321629	best: 0.2321629 (191)	total: 6m 18s	rema

270:	learn: 0.2315487	test: 0.2312280	best: 0.2312280 (270)	total: 9m 20s	remaining: 59m 37s
271:	learn: 0.2315431	test: 0.2312216	best: 0.2312216 (271)	total: 9m 22s	remaining: 59m 34s
272:	learn: 0.2315350	test: 0.2312131	best: 0.2312131 (272)	total: 9m 25s	remaining: 59m 37s
273:	learn: 0.2315290	test: 0.2312070	best: 0.2312070 (273)	total: 9m 27s	remaining: 59m 37s
274:	learn: 0.2315222	test: 0.2312009	best: 0.2312009 (274)	total: 9m 30s	remaining: 59m 37s
275:	learn: 0.2315092	test: 0.2311845	best: 0.2311845 (275)	total: 9m 32s	remaining: 59m 38s
276:	learn: 0.2315031	test: 0.2311769	best: 0.2311769 (276)	total: 9m 35s	remaining: 59m 39s
277:	learn: 0.2314956	test: 0.2311701	best: 0.2311701 (277)	total: 9m 37s	remaining: 59m 37s
278:	learn: 0.2314869	test: 0.2311614	best: 0.2311614 (278)	total: 9m 39s	remaining: 59m 37s
279:	learn: 0.2314774	test: 0.2311524	best: 0.2311524 (279)	total: 9m 42s	remaining: 59m 37s
280:	learn: 0.2314691	test: 0.2311447	best: 0.2311447 (280)	total: 9m 

358:	learn: 0.2309492	test: 0.2305864	best: 0.2305864 (358)	total: 12m 36s	remaining: 57m 35s
359:	learn: 0.2309446	test: 0.2305801	best: 0.2305801 (359)	total: 12m 37s	remaining: 57m 32s
360:	learn: 0.2309396	test: 0.2305750	best: 0.2305750 (360)	total: 12m 39s	remaining: 57m 29s
361:	learn: 0.2309330	test: 0.2305662	best: 0.2305662 (361)	total: 12m 42s	remaining: 57m 30s
362:	learn: 0.2309284	test: 0.2305626	best: 0.2305626 (362)	total: 12m 44s	remaining: 57m 29s
363:	learn: 0.2309245	test: 0.2305590	best: 0.2305590 (363)	total: 12m 47s	remaining: 57m 28s
364:	learn: 0.2309163	test: 0.2305507	best: 0.2305507 (364)	total: 12m 50s	remaining: 57m 30s
365:	learn: 0.2309135	test: 0.2305475	best: 0.2305475 (365)	total: 12m 52s	remaining: 57m 30s
366:	learn: 0.2309086	test: 0.2305411	best: 0.2305411 (366)	total: 12m 55s	remaining: 57m 29s
367:	learn: 0.2309028	test: 0.2305343	best: 0.2305343 (367)	total: 12m 57s	remaining: 57m 26s
368:	learn: 0.2308965	test: 0.2305285	best: 0.2305285 (368)	

...

1577:	learn: 0.2285431	test: 0.2283137	best: 0.2283137 (1577)	total: 59m 34s	remaining: 15m 55s
1578:	learn: 0.2285424	test: 0.2283132	best: 0.2283132 (1578)	total: 59m 37s	remaining: 15m 53s
1579:	learn: 0.2285421	test: 0.2283132	best: 0.2283132 (1578)	total: 59m 39s	remaining: 15m 51s
1580:	learn: 0.2285413	test: 0.2283122	best: 0.2283122 (1580)	total: 59m 41s	remaining: 15m 49s
1581:	learn: 0.2285405	test: 0.2283114	best: 0.2283114 (1581)	total: 59m 43s	remaining: 15m 46s
1582:	learn: 0.2285401	test: 0.2283114	best: 0.2283114 (1582)	total: 59m 45s	remaining: 15m 44s
1583:	learn: 0.2285395	test: 0.2283112	best: 0.2283112 (1583)	total: 59m 47s	remaining: 15m 42s
1584:	learn: 0.2285391	test: 0.2283111	best: 0.2283111 (1584)	total: 59m 49s	remaining: 15m 39s
1585:	learn: 0.2285383	test: 0.2283102	best: 0.2283102 (1585)	total: 59m 52s	remaining: 15m 37s
1586:	learn: 0.2285373	test: 0.2283095	best: 0.2283095 (1586)	total: 59m 55s	remaining: 15m 35s
1587:	learn: 0.2285368	test: 0.2283088	b

1662:	learn: 0.2284855	test: 0.2282743	best: 0.2282743 (1661)	total: 1h 3m 9s	remaining: 12m 47s
1663:	learn: 0.2284851	test: 0.2282741	best: 0.2282741 (1663)	total: 1h 3m 12s	remaining: 12m 45s
1664:	learn: 0.2284808	test: 0.2282696	best: 0.2282696 (1664)	total: 1h 3m 15s	remaining: 12m 43s
1665:	learn: 0.2284807	test: 0.2282696	best: 0.2282696 (1665)	total: 1h 3m 17s	remaining: 12m 41s
1666:	learn: 0.2284778	test: 0.2282669	best: 0.2282669 (1666)	total: 1h 3m 20s	remaining: 12m 39s
1667:	learn: 0.2284770	test: 0.2282662	best: 0.2282662 (1667)	total: 1h 3m 22s	remaining: 12m 36s
1668:	learn: 0.2284764	test: 0.2282661	best: 0.2282661 (1668)	total: 1h 3m 24s	remaining: 12m 34s
1669:	learn: 0.2284757	test: 0.2282660	best: 0.2282660 (1669)	total: 1h 3m 26s	remaining: 12m 32s
1670:	learn: 0.2284752	test: 0.2282658	best: 0.2282658 (1670)	total: 1h 3m 28s	remaining: 12m 29s
1671:	learn: 0.2284748	test: 0.2282657	best: 0.2282657 (1671)	total: 1h 3m 30s	remaining: 12m 27s
1672:	learn: 0.228474

1747:	learn: 0.2284184	test: 0.2282246	best: 0.2282246 (1747)	total: 1h 6m 22s	remaining: 9m 34s
1748:	learn: 0.2284181	test: 0.2282246	best: 0.2282246 (1747)	total: 1h 6m 25s	remaining: 9m 31s
1749:	learn: 0.2284170	test: 0.2282239	best: 0.2282239 (1749)	total: 1h 6m 27s	remaining: 9m 29s
1750:	learn: 0.2284162	test: 0.2282232	best: 0.2282232 (1750)	total: 1h 6m 29s	remaining: 9m 27s
1751:	learn: 0.2284157	test: 0.2282229	best: 0.2282229 (1751)	total: 1h 6m 32s	remaining: 9m 25s
1752:	learn: 0.2284149	test: 0.2282222	best: 0.2282222 (1752)	total: 1h 6m 35s	remaining: 9m 22s
1753:	learn: 0.2284122	test: 0.2282197	best: 0.2282197 (1753)	total: 1h 6m 38s	remaining: 9m 20s
1754:	learn: 0.2284118	test: 0.2282197	best: 0.2282197 (1753)	total: 1h 6m 40s	remaining: 9m 18s
1755:	learn: 0.2284113	test: 0.2282195	best: 0.2282195 (1755)	total: 1h 6m 42s	remaining: 9m 16s
1756:	learn: 0.2284109	test: 0.2282195	best: 0.2282195 (1756)	total: 1h 6m 45s	remaining: 9m 13s
1757:	learn: 0.2284082	test: 0

1832:	learn: 0.2283608	test: 0.2281835	best: 0.2281835 (1832)	total: 1h 9m 34s	remaining: 6m 20s
1833:	learn: 0.2283600	test: 0.2281829	best: 0.2281829 (1833)	total: 1h 9m 36s	remaining: 6m 18s
1834:	learn: 0.2283598	test: 0.2281829	best: 0.2281829 (1834)	total: 1h 9m 39s	remaining: 6m 15s
1835:	learn: 0.2283595	test: 0.2281826	best: 0.2281826 (1835)	total: 1h 9m 41s	remaining: 6m 13s
1836:	learn: 0.2283592	test: 0.2281824	best: 0.2281824 (1836)	total: 1h 9m 43s	remaining: 6m 11s
1837:	learn: 0.2283584	test: 0.2281821	best: 0.2281821 (1837)	total: 1h 9m 46s	remaining: 6m 9s
1838:	learn: 0.2283581	test: 0.2281819	best: 0.2281819 (1838)	total: 1h 9m 48s	remaining: 6m 6s
1839:	learn: 0.2283578	test: 0.2281818	best: 0.2281818 (1839)	total: 1h 9m 50s	remaining: 6m 4s
1840:	learn: 0.2283568	test: 0.2281803	best: 0.2281803 (1840)	total: 1h 9m 53s	remaining: 6m 2s
1841:	learn: 0.2283564	test: 0.2281798	best: 0.2281798 (1841)	total: 1h 9m 55s	remaining: 5m 59s
1842:	learn: 0.2283561	test: 0.228

1917:	learn: 0.2282937	test: 0.2281313	best: 0.2281313 (1917)	total: 1h 12m 55s	remaining: 3m 7s
1918:	learn: 0.2282929	test: 0.2281306	best: 0.2281306 (1918)	total: 1h 12m 57s	remaining: 3m 4s
1919:	learn: 0.2282926	test: 0.2281301	best: 0.2281301 (1919)	total: 1h 12m 59s	remaining: 3m 2s
1920:	learn: 0.2282918	test: 0.2281296	best: 0.2281296 (1920)	total: 1h 13m 1s	remaining: 3m
1921:	learn: 0.2282915	test: 0.2281295	best: 0.2281295 (1921)	total: 1h 13m 3s	remaining: 2m 57s
1922:	learn: 0.2282902	test: 0.2281285	best: 0.2281285 (1922)	total: 1h 13m 5s	remaining: 2m 55s
1923:	learn: 0.2282897	test: 0.2281280	best: 0.2281280 (1923)	total: 1h 13m 8s	remaining: 2m 53s
1924:	learn: 0.2282879	test: 0.2281267	best: 0.2281267 (1924)	total: 1h 13m 11s	remaining: 2m 51s
1925:	learn: 0.2282877	test: 0.2281266	best: 0.2281266 (1925)	total: 1h 13m 13s	remaining: 2m 48s
1926:	learn: 0.2282869	test: 0.2281261	best: 0.2281261 (1926)	total: 1h 13m 15s	remaining: 2m 46s
1927:	learn: 0.2282862	test: 0.

In [53]:
mean_squared_error(y_test, pred)

0.05216318842352748

<h1>Make Submission</h1>

In [54]:
test_df = pd.read_csv('test.csv')
simple_test_df = test_df[['region', 'city', 'parent_category_name', 'category_name', 'price', 
                      'item_seq_number', 'user_type', 'image_top_1']]
simple_test_df.price = simple_test_df.price.fillna(simple_df.price.median())
simple_test_df.image_top_1 = simple_test_df.image_top_1.fillna(simple_df.image_top_1.max()+1)
simple_test_df.image_top_1 = simple_test_df.image_top_1.astype(int)
print(simple_test_df.shape[0])
simple_test_df.isnull().any()

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


508438


region                  False
city                    False
parent_category_name    False
category_name           False
price                   False
item_seq_number         False
user_type               False
image_top_1             False
dtype: bool

In [55]:
def make_prediction(a):
    a = np.maximum(a, np.zeros(a.shape))
    answer = test_df[['item_id']].assign(deal_probability = a)
    answer.to_csv('sample_submission2.csv', sep = ',', index = False, header = True)

In [56]:
pred = kitty.predict(simple_test_df)
make_prediction(pred)